In [ ]:
import cv2
import os
from pathlib import Path
import shutil
import random


In [ ]:
# ----------------  CONFIG  ----------------
# where your current data lives
SOURCE_ROOT = Path("/cats/hi")  # contains cat1_1, cat1_2, cat2_1, ...

OUT_ROOT = Path("id_dataset")
OUT_ROOT.mkdir(exist_ok=True)

PAD = 0.15  # 15% padding around bbox

DST_ROOT = Path("id_dataset_split")
TRAIN_RATIO = 0.8 # train/test split
SEED = 42 # random seed
IMAGE_EXTS = {".jpg", ".jpeg", ".png"}

In [3]:
def yolo_to_xyxy(label, img_w, img_h):
    _, cx, cy, w, h = label
    cx, cy, w, h = map(float, (cx, cy, w, h))

    bw = int(w * img_w)
    bh = int(h * img_h)
    cx = int(cx * img_w)
    cy = int(cy * img_h)

    x1 = cx - bw // 2
    y1 = cy - bh // 2
    x2 = cx + bw // 2
    y2 = cy + bh // 2
    return x1, y1, x2, y2

In [4]:
for folder in SOURCE_ROOT.iterdir():
    if not folder.is_dir():
        continue

    # infer identity from folder name
    if folder.name.startswith("devi"):
        out_class = "devi"
    elif folder.name.startswith("sati"):
        out_class = "sati"
    else:
        continue

    out_dir = OUT_ROOT / out_class
    out_dir.mkdir(exist_ok=True)

    img_dir = folder / "images"
    lbl_dir = folder / "annotations"

    for img_path in img_dir.glob("*.jpg"):
        lbl_path = lbl_dir / (img_path.stem + ".txt")
        if not lbl_path.exists():
            continue

        img = cv2.imread(str(img_path))
        h, w = img.shape[:2]

        with open(lbl_path) as f:
            for i, line in enumerate(f):
                parts = line.strip().split()
                if len(parts) != 5:
                    continue

                x1, y1, x2, y2 = yolo_to_xyxy(parts, w, h)

                # padding
                bw = x2 - x1
                bh = y2 - y1
                px = int(bw * PAD)
                py = int(bh * PAD)

                x1 = max(0, x1 - px)
                y1 = max(0, y1 - py)
                x2 = min(w - 1, x2 + px)
                y2 = min(h - 1, y2 + py)

                crop = img[y1:y2, x1:x2]
                if crop.shape[0] < 80 or crop.shape[1] < 80:
                    continue

                out_name = f"{folder.name}_{img_path.stem}_{i}.jpg"
                cv2.imwrite(str(out_dir / out_name), crop)


In [ ]:
random.seed(SEED)

for split in ["train", "val"]:
    for cls in ["devi", "sati"]:
        (DST_ROOT / split / cls).mkdir(parents=True, exist_ok=True)

for cls_dir in OUT_ROOT.iterdir():
    if not cls_dir.is_dir():
        continue

    images = [
        p for p in cls_dir.iterdir()
        if p.suffix.lower() in IMAGE_EXTS
    ]

    random.shuffle(images)
    split_idx = int(len(images) * TRAIN_RATIO)

    train_imgs = images[:split_idx]
    val_imgs = images[split_idx:]

    for img in train_imgs:
        shutil.copy2(img, DST_ROOT / "train" / cls_dir.name / img.name)

    for img in val_imgs:
        shutil.copy2(img, DST_ROOT / "val" / cls_dir.name / img.name)

    print(f"{cls_dir.name}: {len(train_imgs)} train / {len(val_imgs)} val")

print("✔ Dataset split complete")


devi: 690 train / 173 val
sati: 690 train / 173 val
✔ Dataset split complete
